In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import simplejson

In [2]:
from clean import split_score,split_review,run

In [86]:
df = run("dianping_info_test.csv","dianping_review_test.csv")
df.columns

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df["no_queues%"] = df["no_queues"]/df["total_review_number"]
df.head()

In [ ]:
df.tag1.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from string import ascii_letters
import numpy as np


In [ ]:
plt.figure(figsize=(20, 4))

plt.subplot(1, 4, 1)
plt.title('Total Score')
sns.histplot(df['total_review_score'], kde=True,bins=10)

plt.subplot(1, 4, 2)
plt.title('Taste Score')
sns.histplot(df['taste'], kde=True,bins=10, color='orange')

plt.subplot(1, 4, 3)
plt.title('Surroundings Score')
sns.histplot(df['surroundings'], kde=True, color='r',bins=10)

plt.subplot(1, 4, 4)
plt.title('Service Score')
sns.histplot(df['service'], kde=True, color='g',bins=10);

In [ ]:
df.dtypes

In [ ]:
df_analysis = df.drop(columns=['restaurant_id','address','tag1','tag2','recommendation_1','recommendation_2','recommendation_3','restaurant'])
df_analysis.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_analysis)
df_transformed = pd.DataFrame(scaler.transform(df_analysis), columns = df_analysis.columns)
df_transformed.head()

In [ ]:
corrMatrix = round(df_transformed.corr(),2)
plt.figure(figsize=(14, 10))
plt.title("Features Correlation",fontsize=20)
mask = np.triu(np.ones_like(corrMatrix, dtype=bool))
cmap = sns.color_palette("mako", as_cmap=True)
sns.heatmap(corrMatrix, annot=True, cmap=cmap, mask=mask)
plt.show();

In [ ]:
# Try different classification techniques
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

linear = LinearRegression()

In [ ]:
df_features = df_transformed.drop('total_review_score',axis = 1)
df_target = df_transformed['total_review_score']

X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size = 0.2, random_state = 10,\
                                                    stratify = df_target)

In [ ]:
linear.fit(X_train,list(y_train.values))
y_pred = linear.predict(X_test)
print(f'model score on training data: {linear.score(X_train, y_train)}')
print(f'model score on testing data: {linear.score(X_test, y_test)}')

In [ ]:
from sklearn.linear_model import RidgeCV

ridge = RidgeCV()
ridge.fit(X_train, y_train)

print(f'model score on training data: {ridge.score(X_train, y_train)}')
print(f'model score on testing data: {ridge.score(X_test, y_test)}')

In [ ]:
df_info = pd.read_excel("raw data/dianping_info.xlsx")
df_info = split_score(df_info)

In [ ]:
dishes_list=df_info['推荐菜'].tolist()
dishes_list[1]

In [ ]:
#3，分割组合成新列表
dishes_list_n=[]
for j in range(0,len(dishes_list)):
    for i in dishes_list[j].split(' '):
        dishes_list_n.append(i)
#4,清洗新列表的标点符号，便于后续排序计数
for i in range(0,len(dishes_list_n)):
    dishes_list_n[i]=dishes_list_n[i].replace("'"," ")
    dishes_list_n[i]=dishes_list_n[i].replace("["," ")
    dishes_list_n[i]=dishes_list_n[i].replace("]"," ")
    dishes_list_n[i]=dishes_list_n[i].replace(" ","")
    dishes_list_n[i]=dishes_list_n[i].replace(",","")
    dishes_list_n[i]=dishes_list_n[i].replace("……","")
#5,列表排序
dishes_list_n.sort()
#6,统计次数
dish_times=pd.value_counts(dishes_list_n).reset_index().rename(columns={'index':'推荐餐点',0:'被推荐次数'})
dish_times

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator

# Start with one review:

text=" ".join(i for i in dish_times['推荐餐点'])

# Create and generate a word cloud image:
wordcloud = WordCloud().generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [4]:
import pandas as pd
import jieba
from matplotlib import pyplot as plt

data = pd.read_csv('raw_data/data1.csv')
data.head()

,cus_id,comment_time,comment_star,cus_comment,kouwei,huanjing,fuwu,shopID,stars,year,month,weekday,hour,comment_len
0,迷糊泰迪,2018/9/20 6:48,sml-str40,南信 算是 广州 著名 甜品店 吧 好几个 时间段 路过 都 是 座无虚席 看着 餐单 上 ...,非常好,好,好,518986,4.0,2018,9,3,6,184
1,稱霸幼稚園,2018/9/22 21:49,sml-str40,中午 吃 完 了 所谓 的 早茶 回去 放下 行李 休息 了 会 就 来 吃 下午茶 了 服...,很好,很好,很好,518986,4.0,2018,9,5,21,266
2,爱吃的美美侠,2018/9/22 22:16,sml-str40,冲刺 王者 战队 吃遍 蓉城 战队 有 特权 五月份 和 好 朋友 毕业 旅行 来 了 广州...,很好,很好,很好,518986,4.0,2018,9,5,22,341
3,姜姜会吃胖,2018/9/19 6:36,sml-str40,都 说来 广州 吃 糖水 就要 来南信 招牌 姜撞奶 红豆 双皮奶 牛 三星 云吞面 一楼 ...,非常好,很好,很好,518986,4.0,2018,9,2,6,197
4,forevercage,2018/8/24 17:58,sml-str50,一直 很 期待 也 最 爱 吃 甜品 广州 的 甜品 很 丰富 很 多样 来 之前 就 一直...,非常好,很好,很好,518986,5.0,2018,8,4,17,261


In [5]:
def zhuanhuan(score):
    if score > 3:
        return 1
    elif score < 3:
        return 0
    else:
        return None

In [6]:
data['target'] = data['stars'].map(lambda x:zhuanhuan(x))
data_model = data.dropna()

In [7]:
#切分测试集、训练集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_model['cus_comment'], data_model['target'], random_state=3, test_size=0.25)

#引入停用词
infile = open("raw_data/cn_stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

#中文分词
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df
 
x_train[0]

'南信 算是 广州 著名 甜品店 吧 好几个 时间段 路过 都 是 座无虚席 看着 餐单 上 密密麻麻 满满当当 好吃 的 容易 范 选择 困难 症拼桌 的 体验 不是 最好 人多 嘈杂 只想 快点 吃 完 赶紧 撤 自己 到 收银台 点单 服务员 只管 上菜 食物 味道 还是 要点 赞 的'

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=3000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['$', '0', '1', '2', '3', '4', '5', '6', '7', '8',
                            '9', '?', '_', '“', '”', '、', '。', '《', '》', '一',
                            '一些', '一何', '一切', '一则', '一方面', '一旦', '一来', '一样',
                            '一般', '一转眼', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, f1_score
classifier = MultinomialNB()
classifier.fit(tv.transform(x_train), y_train)
classifier.score(tv.transform(x_test), y_test)

0.9272794389073459

In [10]:
y_pred = classifier.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred)

0.9094499777254503

In [11]:
def ceshi(model,strings):
    strings_fenci = fenci(pd.Series([strings]))
    return round(float(model.predict_proba(tv.transform(strings_fenci))[:,1]),2)

In [12]:
test1 = '很好吃，环境好，所有员工的态度都很好，上菜快，服务也很好，味道好吃，都是用蒸馏水煮的，推荐，超好吃' #5星好评
test2 = '糯米外皮不绵滑，豆沙馅粗躁，没有香甜味。12元一碗不值。' #1星差评
print('好评实例的模型预测情感得分为{}\n差评实例的模型预测情感得分为{}'.format(ceshi(classifier,test1),ceshi(classifier,test2)))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.230 seconds.
Prefix dict has been built successfully.


好评实例的模型预测情感得分为0.87
差评实例的模型预测情感得分为0.81


In [95]:
data_info =  pd.read_csv("raw_data/dianping_info_test.csv",encoding='utf-8')
data_info.head()

,_id,人均价格,图片链接,店铺id,店铺名,店铺地址,店铺均分,店铺总分,推荐菜,标签1,标签2,评论总数,详情链接
0,62db5b0c817bf9b8a570751f,￥97,http://p0.meituan.net/biztone/179942316_164204...,l1KkGwL9nhdbxYJG,韩聚场石板烤肉(人广世茂店),-,-,4.5,-,韩式烤肉,人民广场/南京路,4529条评价,https://www.dianping.com/shop/l1KkGwL9nhdbxYJG
1,62db5bd3817bf9b8a5707521,￥108,https://img.meituan.net/msmerchant/d1640dfc80e...,G6lUt8p6xQvkx4jf,韩国欧巴炭烤肉오빠숯불구이,-,-,4.5,-,韩式烤肉,虹桥镇,13773条评价,https://www.dianping.com/shop/G6lUt8p6xQvkx4jf
2,62db5c97817bf9b8a5707523,￥118,https://img.meituan.net/msmerchant/0f85cc7577f...,l26n8GRljBs37jw0,齐齐哈尔碳烤肉(江宁路店),-,-,4.0,-,烧烤烤串,长寿路,985条评价,https://www.dianping.com/shop/l26n8GRljBs37jw0
3,62db5d63817bf9b8a5707525,￥108,http://p1.meituan.net/biztone/192843022_165837...,H1AqU2oV2LZE1Eiw,御殿场烧肉(南京西路店),-,-,4.0,-,烧烤烤串,南京西路,4292条评价,https://www.dianping.com/shop/H1AqU2oV2LZE1Eiw
4,62db5e2b817bf9b8a5707527,￥82,https://img.meituan.net/msmerchant/3ac573e4709...,k7hEx6fVSx6Z75W2,圣火营自助烤肉(南京东路店),-,-,4.0,-,烧烤烤串,南京东路,16824条评价,https://www.dianping.com/shop/k7hEx6fVSx6Z75W2


In [96]:
d3 = pd.read_csv("raw_data/dianping_review_test.csv",encoding='utf-8')
d3.head()

,_id,中评个数,好评个数,差评个数,带图评论个数,店铺id,推荐菜,精选评论,评论总数,评论摘要
0,62db5b0c817bf9b8a5707520,567,3513,226,3191,l1KkGwL9nhdbxYJG,-,"[{""店铺id"": ""l1KkGwL9nhdbxYJG"", ""评论id"": ""1109187...",4306,"[{""描述"": ""菜品健康"", ""个数"": ""310""}, {""描述"": ""不用排队"", ""..."
1,62db5bd3817bf9b8a5707522,1123,11613,327,5205,G6lUt8p6xQvkx4jf,-,"[{""店铺id"": ""G6lUt8p6xQvkx4jf"", ""评论id"": ""1116490...",13063,"[{""描述"": ""上菜快"", ""个数"": ""168""}, {""描述"": ""不用排队"", ""个..."
2,62db5c97817bf9b8a5707524,91,777,105,674,l26n8GRljBs37jw0,-,"[{""店铺id"": ""l26n8GRljBs37jw0"", ""评论id"": ""1117401...",973,"[{""描述"": ""牛肉赞"", ""个数"": ""49""}, {""描述"": ""不用排队"", ""个数..."
3,62db5d63817bf9b8a5707526,543,3353,247,3108,H1AqU2oV2LZE1Eiw,-,"[{""店铺id"": ""H1AqU2oV2LZE1Eiw"", ""评论id"": ""1117591...",4143,"[{""描述"": ""菜品健康"", ""个数"": ""212""}, {""描述"": ""不用排队"", ""..."
4,62db5e2b817bf9b8a5707528,1409,6343,634,2457,k7hEx6fVSx6Z75W2,-,"[{""店铺id"": ""k7hEx6fVSx6Z75W2"", ""评论id"": ""1117017...",8386,"[{""描述"": ""菜品健康"", ""个数"": ""337""}, {""描述"": ""不用排队"", ""..."


In [90]:
d3['精选评论'] = d3['精选评论'].map(lambda x:simplejson.loads(x))

In [97]:
# use explode to give each dict in a list a separate row
df_review = pd.DataFrame(d3.精选评论.explode().reset_index(drop=True))

# normalize the column of dicts, join back to the remaining dataframe columns, and drop the unneeded column
df_review = pd.json_normalize(df_review.精选评论)
df_review_NLP = df_review[['评论内容','用户总分','店铺id']]

KeyError: "None of [Index(['评论内容', '用户总分', '店铺id'], dtype='object')] are in the [columns]"

In [35]:
df_review_NLP['评论内容'] = df_review_NLP['评论内容'].apply(lambda x:' '.join(jieba.cut(x)))

/tmp/ipykernel_13870/2206967058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_NLP['评论内容'] = df_review_NLP['评论内容'].apply(lambda x:' '.join(jieba.cut(x)))


In [36]:
# cols = ['用户总分','用户打分.口味','用户打分.环境','用户打分.服务']
cols = ['用户总分']

for i in cols:
    df_review_NLP[i] = pd.to_numeric(df_review_NLP[i],errors='coerce')

/tmp/ipykernel_13870/3972556165.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_NLP[i] = pd.to_numeric(df_review_NLP[i],errors='coerce')


In [37]:
df_review_NLP['target'] = df_review_NLP['用户总分'].map(lambda x:score_trans(x))
df_model = df_review_NLP.dropna()

/tmp/ipykernel_13870/1501798206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_NLP['target'] = df_review_NLP['用户总分'].map(lambda x:score_trans(x))


In [38]:
x_train, x_test, y_train, y_test = train_test_split(df_model['评论内容'], df_model['target'], random_state=3, test_size=0.25)

In [39]:
#引入停用词
infile = open("raw_data/cn_stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

In [40]:
tv = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=3000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['$', '0', '1', '2', '3', '4', '5', '6', '7', '8',
                            '9', '?', '_', '“', '”', '、', '。', '《', '》', '一',
                            '一些', '一何', '一切', '一则', '一方面', '一旦', '一来', '一样',
                            '一般', '一转眼', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [41]:
classifier = MultinomialNB()
classifier.fit(tv.transform(x_train), y_train)
classifier.score(tv.transform(x_test), y_test)

0.9223671651288577

In [42]:
y_pred = classifier.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred)

0.8762372223589145

In [43]:
test1 = '很好吃，环境好，所有员工的态度都很好，上菜快，服务也很好，味道好吃，都是用蒸馏水煮的，推荐，超好吃' #5星好评
test2 = '糯米外皮不绵滑，豆沙馅粗躁，没有香甜味。12元一碗不值，不好吃。' #1星差评
print('好评实例的模型预测情感得分为{}\n差评实例的模型预测情感得分为{}'.format(ceshi(classifier,test1),ceshi(classifier,test2)))

好评实例的模型预测情感得分为0.95
差评实例的模型预测情感得分为0.79


In [62]:
df_review_NLP['target'].value_counts()

1.0    11581
0.0      988
Name: target, dtype: int64

In [72]:
index_tmp = y_train==0
y_tmp = y_train[index_tmp]
x_tmp = x_train[index_tmp]
x_train2 = pd.concat([x_train,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp])
y_train2 = pd.concat([y_train,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp])

In [73]:
clf2 = MultinomialNB()
clf2.fit(tv.transform(x_train2), y_train2)
y_pred2 = clf2.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred2)

0.8920289328293108

In [74]:
print(ceshi(clf2,'很好吃，环境好，超好吃'))
print(ceshi(clf2,'排队人太多，环境不好，不好吃'))

0.81
0.69


In [48]:
from nlp import nlp_score,review_clean,score_trans,test

In [49]:
pd.read_csv("raw_data/dianping_review_test.csv",encoding='utf-8').head()

,_id,中评个数,好评个数,差评个数,带图评论个数,店铺id,推荐菜,精选评论,评论总数,评论摘要
0,62db5b0c817bf9b8a5707520,567,3513,226,3191,l1KkGwL9nhdbxYJG,-,"[{""店铺id"": ""l1KkGwL9nhdbxYJG"", ""评论id"": ""1109187...",4306,"[{""描述"": ""菜品健康"", ""个数"": ""310""}, {""描述"": ""不用排队"", ""..."
1,62db5bd3817bf9b8a5707522,1123,11613,327,5205,G6lUt8p6xQvkx4jf,-,"[{""店铺id"": ""G6lUt8p6xQvkx4jf"", ""评论id"": ""1116490...",13063,"[{""描述"": ""上菜快"", ""个数"": ""168""}, {""描述"": ""不用排队"", ""个..."
2,62db5c97817bf9b8a5707524,91,777,105,674,l26n8GRljBs37jw0,-,"[{""店铺id"": ""l26n8GRljBs37jw0"", ""评论id"": ""1117401...",973,"[{""描述"": ""牛肉赞"", ""个数"": ""49""}, {""描述"": ""不用排队"", ""个数..."
3,62db5d63817bf9b8a5707526,543,3353,247,3108,H1AqU2oV2LZE1Eiw,-,"[{""店铺id"": ""H1AqU2oV2LZE1Eiw"", ""评论id"": ""1117591...",4143,"[{""描述"": ""菜品健康"", ""个数"": ""212""}, {""描述"": ""不用排队"", ""..."
4,62db5e2b817bf9b8a5707528,1409,6343,634,2457,k7hEx6fVSx6Z75W2,-,"[{""店铺id"": ""k7hEx6fVSx6Z75W2"", ""评论id"": ""1117017...",8386,"[{""描述"": ""菜品健康"", ""个数"": ""337""}, {""描述"": ""不用排队"", ""..."


In [50]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import jieba
import simplejson
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, f1_score

In [51]:
df = review_clean(pd.read_csv("raw_data/dianping_review_test.csv"))
df.head()

,评论内容,用户总分,店铺id
0,平台上买的他家的双人套餐，真的非常非常实在，份量足，味道也很好，推荐推荐 他家是自助铁板烤...,5.0,l1KkGwL9nhdbxYJG
1,三月份就中的霸王餐被疫情耽搁到现在才去吃真痛苦 这家韩聚场平时也常吃，特色的石板烤肉蛮少见的...,5.0,l1KkGwL9nhdbxYJG
2,呼呼离谱三人行，人均92吃成了143哈哈哈哈 场地很大，没排队就直接入座啦～服务员也很麻利...,5.0,l1KkGwL9nhdbxYJG
3,刷某音看到就团了一个随便挑个日子就去了周二6点多店里人不多套餐里有4种肉肉一个菇一份炒年糕还...,4.5,l1KkGwL9nhdbxYJG
4,解封之后的第一顿烤肉！用餐感受还是非常满意的，可能正好也是赶上肚子饿，吃的可香了，哈哈～肉都...,4.5,l1KkGwL9nhdbxYJG


In [104]:
nlp_score("raw_data/dianping_review_test.csv")

0.9328791684014021


,评论内容,用户总分,店铺id,target,nlp_score
0,平台 上 买 的 他家 的 双人 套餐 ， 真的 非常 非常 实在 ， 份量 足 ， 味道 ...,5.0,l1KkGwL9nhdbxYJG,1.0,0.97
1,三月份 就 中 的 霸王餐 被 疫情 耽搁到 现在 才 去 吃 真 痛苦 这家 韩聚场 ...,5.0,l1KkGwL9nhdbxYJG,1.0,0.94
2,呼呼 离谱 三人行 ， 人均 92 吃成 了 143 哈哈哈哈 场地 很大 ， 没 ...,5.0,l1KkGwL9nhdbxYJG,1.0,0.94
3,刷 某音 看到 就团 了 一个 随便 挑个 日子 就 去 了 周二 6 点多 店里 人不多 ...,4.5,l1KkGwL9nhdbxYJG,1.0,0.97
4,解封 之后 的 第一顿 烤肉 ！ 用餐 感受 还是 非常 满意 的 ， 可能 正好 也 是 ...,4.5,l1KkGwL9nhdbxYJG,1.0,0.91
...,...,...,...,...,...
12985,超级 好吃 的 火锅 ！ 不 知道 为啥 就是 觉得 很 好吃,5.0,Ga7fXeKdJ29SZjZO,1.0,0.80
12986,因为 疫情 的 缘故 ， 导致 这个 团购 卷 过期 了 一天 ， 好 在 提前 加微 信验...,4.0,Ga7fXeKdJ29SZjZO,1.0,0.76
12987,没想到 公司 附近 开 了 一家 哈哈哈 真 好 菜品 还是 一如既往 的 好吃 买 了...,5.0,Ga7fXeKdJ29SZjZO,1.0,0.98
12988,他家 锅底 不能 选择 辣度 ， 我 自认 吃 辣 不算 菜鸡 ， 哥老官 的 中辣 吃 了...,4.0,Ga7fXeKdJ29SZjZO,1.0,0.71
